In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
path= "/kaggle/input/fruit-images-for-object-detection/train_zip/train"
fileslst= os.listdir(path); imgs= [];
for fle in fileslst:
    if fle.endswith(".jpg"):
        imgs.append(fle)
#print(imgs)
img= cv2.imread("/kaggle/input/fruit-images-for-object-detection/train_zip/train/"+imgs[1]); 
img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
img= cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
plt.subplot(231); plt.imshow(img[:,:,0])
plt.subplot(232); plt.imshow(img[:,:,1])
plt.subplot(233); plt.imshow(img[:,:,2])
imgedg= []
imgedg0= cv2.Canny(img[:,:,0],120,200); imgedg1= cv2.Canny(img[:,:,1],120,200); imgedg2= cv2.Canny(img[:,:,2],120,200);
plt.subplot(234); plt.imshow(imgedg0)
plt.subplot(235); plt.imshow(imgedg1)
plt.subplot(236); plt.imshow(imgedg2)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path= "/kaggle/input/fruit-images-for-object-detection/train_zip/train"
fileslst= os.listdir(path); imgs= [];
for fle in fileslst:
    if fle.endswith(".jpg"):
        imgs.append(fle)
imgar= []; y= pd.DataFrame(columns=["apl","ban","orn","mix"])
print(fileslst[:20])
for i in range(len(imgs)):
    img= cv2.imread("/kaggle/input/fruit-images-for-object-detection/train_zip/train/"+imgs[i]); 
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(28,28))
    imgar.append(np.array(img))
    if imgs[i][0]=='a':
        y.loc[i,"apl"]=1
    elif imgs[i][0]=='b':
        y.loc[i,"ban"]=1
    elif imgs[i][0]=='o':
        y.loc[i,"orn"]=1
    else:
        y.loc[i,"mix"]=1

imgarr= np.array([imgar]); imgarr= np.reshape(imgarr,(240,1,28,28,3))
print(np.shape(imgarr))
y= y.replace(np.nan,0)
y= np.array([y]); y= np.reshape(y,(240,4))
print(y[0])
plt.imshow(img)
img= cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
plt.subplot(231); plt.imshow(img[:,:,0])
plt.subplot(232); plt.imshow(img[:,:,1])
plt.subplot(233); plt.imshow(img[:,:,2])
imgedg= []
imgedg0= cv2.Canny(img[:,:,0],120,200); imgedg1= cv2.Canny(img[:,:,1],120,200); imgedg2= cv2.Canny(img[:,:,2],120,200);
plt.subplot(234); plt.imshow(imgedg0)
plt.subplot(235); plt.imshow(imgedg1)
plt.subplot(236); plt.imshow(imgedg2)

In [ ]:
modl= tf.keras.Sequential()
modl.add(tf.keras.layers.InputLayer(input_shape=(1,28,28,3)))
modl.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu', name="convlyr"))
modl.add(tf.keras.layers.Flatten())
modl.add(tf.keras.layers.Dense(units= 64, activation='relu'))
modl.add(tf.keras.layers.Dense(32, activation="relu"))

modl.add(tf.keras.layers.Dense(4,activation="sigmoid"))
         
modl.summary()
optm= tf.keras.optimizers.Adam(learning_rate=0.001)
modl.compile(loss= 'categorical_crossentropy', optimizer= optm, metrics= 'accuracy')

In [ ]:
modl.fit(imgarr,y, epochs= 50)

In [ ]:
modl.save("/kaggle/working/frutclssfrsig.h5")

In [ ]:
print(np.shape(imgarr))
#yprd= modl.predict(np.array([imgarr[0]]))
#print(yprd)
img= cv2.imread("/kaggle/input/fruit-images-for-object-detection/test_zip/test/"+"banana_82.jpg"); 
#img= cv2.imread("/kaggle/input/appletest1/banapl.jpg")
img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img= cv2.resize(img,(28,28))
xnw= np.array([[img]]); print(np.shape(xnw))
yprd= modl.predict(xnw)
print(yprd)

In [ ]:
import matplotlib.pyplot as plt
lyrop= modl.get_layer('convlyr').output
lyrmdl= tf.keras.models.Model(inputs = modl.input, outputs = lyrop)
lyrpred= lyrmdl.predict(xnw); plt.subplot(221);plt.imshow(img); plt.subplot(222);
plt.imshow(lyrpred[0,0,:,:,0]); plt.subplot(223);plt.imshow(lyrpred[0,0,:,:,6]); plt.subplot(224);plt.imshow(lyrpred[0,0,:,:,8]); 

functional api to implement residual network that include color features

In [ ]:
#funcional model with residual neural network
inp= tf.keras.Input(shape=(1,28,28,3));
x= tf.keras.layers.Conv2D(16,(3,3),activation='relu')(inp);
x1= tf.keras.layers.AveragePooling2D(pool_size=(3,3))(inp[:,0,:,:,:]);
x1= tf.keras.layers.Dropout(0.2)(x1)
#x1= tf.expand_dims(x1, axis=0)
x= tf.keras.layers.Flatten()(x)
x1= tf.keras.layers.Flatten()(x1)
combind= tf.keras.layers.concatenate([x,x1])
x= tf.keras.layers.Dense(units= 32, activation='relu')(combind)
x= tf.keras.layers.Dense(16, activation="relu")(x)
out= tf.keras.layers.Dense(4,activation="softmax")(x)
modl1= tf.keras.Model(inputs=inp, outputs=out)
                                
modl1.summary()
optm= tf.keras.optimizers.Adam(learning_rate=0.001)
modl1.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy','TrueNegatives'])

In [ ]:
modl1.fit(imgarr,y, epochs= 60)

In [ ]:
print(np.shape(imgarr))
#yprd= modl.predict(np.array([imgarr[0]]))
#print(yprd)
img= cv2.imread("/kaggle/input/fruit-images-for-object-detection/test_zip/test/"+"banana_82.jpg"); 
img= cv2.imread("/kaggle/input/appletest/banapl.jpg")
img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img= cv2.resize(img,(28,28))
xnw= np.array([[img]]); print(np.shape(xnw))
yprd= modl1.predict(xnw)
print(yprd)

In [ ]:
modl1.save("/kaggle/ouput/fruitclsswithcolr.h5")